## Tuning and deploy a foundation model

Creating an LLM requires massive amounts of data, significant computing resources, and specialized skills. On Vertex AI, tuning allows you to customize a foundation model for more specific tasks or knowledge domains.

While the prompt design is excellent for quick experimentation, if training data is available, you can achieve higher quality by tuning the model. Tuning a model enables you to customize the model response based on examples of the task you want the model to perform.

For more details on tuning have a look at the [official documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/models/tune-models).

### Quota
**important**: Tuning the text-bison@001  model uses the tpu-v3-8 training resources and the accompanying quotas from your Google Cloud project. Each project has a default quota of eight v3-8 cores, which allows for one to two concurrent tuning jobs. If you want to run more concurrent jobs you need to request additional quota via the [Quotas page](https://console.cloud.google.com/iam-admin/quotas).

### Install Vertex AI SDK

In [ ]:
!pip install google-cloud-aiplatform google-cloud-bigquery sequence-evaluate sentence-transformers rouge --upgrade --user

### Set your project ID

In [ ]:
PROJECT_ID = "gen-id-ai-canada"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

### Create a bucket
Now you have to create a bucket that we will use to store our tuning data. To avoid name collisions between users on resources created, you generate a UUID for each instance session and append it to the name of the resources you create in this tutorial.

In [ ]:
import random
import string
import os
print(os.getcwd())

# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))

UUID = generate_uuid()

Choose a bucket name and update the `BUCKET_NAME` parameter.

In [ ]:
BUCKET_NAME = "warren_genai_bucket"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"
REGION = "us-central1"  # @param {type: "string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = "vertex-" + UUID
    BUCKET_URI = f"gs://{BUCKET_NAME}"

Only if your bucket doesn't already exist: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al $BUCKET_URI

### Import libraries

In [ ]:
from typing import Union

import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

from vertexai.preview.language_models import TextGenerationModel
from google.cloud import aiplatform
from google.cloud import bigquery

## Tune your Model

Now it's time for you to create a tuning job. Tune a foundation model by creating a pipeline job using Generative AI Studio, cURL, or the Python SDK. In this notebook, we will be using the Python SDK. You will be using a Q&A with a context dataset in JSON format.

### Training Data
💾 Your model tuning dataset must be in a JSONL format where each line contains a single training example. You must make sure that you include instructions.

Our data input consists on human crafted and designed prompts

In [ ]:
df = pd.read_csv ("/home/jupyter/warren_genai/data/promps_warren_genai_1.csv")
warren_designed_propmts = df.to_json (r'/home/jupyter/warren_genai/data/promps_warren_genai_1.json')

In [ ]:
df.head()

In [ ]:
print(len(df))

Lets split the data into training and evalation. For Extractive Q&A tasks we advise 100+ training examples. In this case you will use 800.

In [ ]:
# split is set to 80/20
train, evaluation = train_test_split(df, test_size=0.2)
print(len(train))

For tuning, the training data first needs to be converted into a JSONL format.

In [ ]:
tune_jsonl = train.to_json(orient='records', lines=True)

print(f"Length: {len(tune_jsonl)}")
print(tune_jsonl[0:100])

Next, you can write it to a local JSONL before transferring it to Google Cloud Storage (GCS).

In [ ]:
training_data_filename = "tune_data_stack_overflow_warren_genai.jsonl"

with open(training_data_filename, "w") as f:
    f.write(tune_jsonl)

You can then export the local file to GCS, so that it can be used by Vertex AI for the tuning job.

In [ ]:
! gsutil cp $training_data_filename $BUCKET_URI

You can check to make sure that the file successfully transferred to your Google Cloud Storage bucket:

In [ ]:
! gsutil ls -al $BUCKET_URI

In [ ]:
TRAINING_DATA_URI = f"{BUCKET_URI}/{training_data_filename}"

### Model Tuning
Now it's time to start to tune a model. You will use the Vertex AI SDK to submit our tuning job.

#### Recommended Tuning Configurations
✅ Here are some recommended configurations for tuning a foundation model based on the task, in this example Q&A. You can find more in the [documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/models/tune-models).

Extractive QA:
- Make sure that your train dataset size is 100+
- Training steps [100-500]. You can try more than one value to get the best performance on a particular dataset (e.g. 100, 200, 500)

In [ ]:
MODEL_NAME= f"warren-genai-1-tuned-model-{UUID}"

In [ ]:
#Function that starts the tuning job
def tuned_model(
    project_id: str,
    location: str,
    training_data: str,
    model_display_name: str,
    train_steps = 100,
):

    """Prompt-tune a new model, based on a prompt-response data.

    "training_data" can be either the GCS URI of a file formatted in JSONL format
    (for example: training_data=f'gs://{bucket}/{filename}.jsonl'), or a pandas
    DataFrame. Each training example should be JSONL record with two keys, for
    example:
      {
        "input_text": <input prompt>,
        "output_text": <associated output>
      },

    Args:
      project_id: GCP Project ID, used to initialize aiplatform
      location: GCP Region, used to initialize aiplatform
      training_data: GCS URI of training file or pandas dataframe of training data
      model_display_name: Name for your model. 
      train_steps: Number of training steps to use when tuning the model
    """

    aiplatform.init(project=project_id, location=location)
    model = TextGenerationModel.from_pretrained("text-bison@001")

    model.tune_model(
        training_data=training_data,
        model_display_name=model_display_name,
        train_steps=train_steps,
        # Tuning can only happen in the "europe-west4" location
        tuning_job_location="europe-west4",
        # Model can only be deployed in the "us-central1" location
        tuned_model_location="us-central1",
    )

    # Test the tuned model:
    print(model.predict("Recommend an investment strategy"))

    return model

Next it's time to start your tuning job. **Disclaimer:** tuning and deploying a model takes time.

In [ ]:
# This will start the tuning job and output a URL where you can monitor the pipeline execution.
model = tuned_model(PROJECT_ID, REGION, TRAINING_DATA_URI, MODEL_NAME)

Following the link above, you can view your pipeline run. As you can see in the screenshot below, it will execute the following steps:

- Validation
- Export managed dataset
- Convert JSONL to TFRecord
- Large language model tuning
- Upload LLM Model

## View your tuned foundational model on Vertex AI Model registry
When your tuning job is finished, your model will be available on Vertex AI Model Registry. The following Python SDK sample shows you how to list tuned models.

In [ ]:
def list_tuned_models(project_id, location):

    aiplatform.init(project=project_id, location=location)
    model = TextGenerationModel.from_pretrained("text-bison@001")
    tuned_model_names = model.list_tuned_model_names()
    print(tuned_model_names)

In [ ]:
list_tuned_models(PROJECT_ID, REGION)

You can also use the Google Cloud Console UI to view all of your model in [Vertex AI Model Registry](https://console.cloud.google.com/vertex-ai/models?e=13802955&jsmode=O&mods=-ai_platform_fake_service&project=cloud-llm-preview1). Below you can see an example of a tuned foundational model available on Vertex AI Model Registry.

## Use your tuned model to get predictions
Now it's time to get predictions. First you need to get the latest tuned model from the Vertex AI Model registry.

In [ ]:
def fetch_model(project_id, location):

    aiplatform.init(project=project_id, location=location)
    model = TextGenerationModel.from_pretrained("text-bison@001")
    list_tuned_models = model.list_tuned_model_names()
    tuned_model = list_tuned_models[0]

    return tuned_model

In [ ]:
deployed_model = fetch_model(PROJECT_ID, REGION)
deployed_model = TextGenerationModel.get_tuned_model(deployed_model)

Now you can start send a prompt to the API. Feel free to update the following prompt.

In [ ]:
PROMPT = """
You are financial investment assistant, knowledgeable about investment strategies, 
risk profiles and portfolio recommendations.
Provide financial advice based on user age, occupation, risk tolerance, budget for investment, 
investment goals

"""

In [ ]:
print(deployed_model.predict(PROMPT))

## Evaulation
It's essential to evaluate your model to understand its performance. Evaluation can be done in an automated way using evaluation metrics like F1 or Rouge. You can also leverage human evaluation methods. Human evaluation methods involve asking humans to rate the quality of the LLM's answers. This can be done through crowdsourcing or by having experts evaluate the responses. Some standard human evaluation metrics include fluency, coherence, relevance, and informativeness. Often you want to choose a mix of evaluation metrics to get a good understanding of your model performance. Below you will find an example of how you can do the evaluation.

In this example you will be using [sequence-evaluate](https://pypi.org/project/sequence-evaluate/) to evaluation the tuned model.

In [ ]:
from seq_eval import SeqEval
evaluator = SeqEval()

Earlier in the notebook, you created a train and eval dataset. Now it's time to take some of the eval data. You will use the questions to get a response from our tuned model, and the answers we will use as a reference:

- **Candidates**: Answers generated by the tuned model.
- **References**: Original answers that we will use to compare.

In [ ]:
evaluation = evaluation.head(10) # you can change the number of rows you want to use
evaluation_question = evaluation["input_text"]
evaluation_answer = evaluation["output_text"]

Now you can go ahead and generate candidates using the tuned model based on the questions you took from the eval dataset.

In [ ]:
candidates = []

for i in evaluation_question:
    response = deployed_model.predict(i)
    candidates.append(response.text)

len(candidates)

You will also have to create a list of our references. These will you use to evaluate the model's performance.

In [ ]:
references = evaluation_answer.tolist()

len(references)

Next you will generate the evaluation metrics. `evaluator.evaluate` will return a few eval metrics. Some of the important ones are:
- [Blue](https://en.wikipedia.org/wiki/BLEU): The BLEU evaluation metric is a measure of the similarity between a machine-generated text and a human-written reference text.
- [Rouge](https://en.wikipedia.org/wiki/ROUGE_(metric)): The ROUGE evaluation metric is a measure of the overlap between a machine-generated text and a human-written reference text.

In [ ]:
scores = evaluator.evaluate(candidates, references, verbose=False)
print(scores)